In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import time
import re
import numpy as np
import pandas as pd
# import undetected_chromedriver.v2 as uc2
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import gc

In [2]:
df = pd.read_csv("smartphones_missing_imputation_v1.csv")
df.head()

,name,link,price,rating,spec_score,processor_brand,processor_core,clock_speed,has_ir,has_nfc,...,ppi,num_rear_cameras,num_front_cameras,rear_primary_mp,front_primary_mp,expandable_memory_supported,expandable_memory_limit,price_group,battery_life_group,ram_gb_group
0,Doogee S119,https://www.smartprix.com/mobiles/doogee-s119-...,29999.0,4.40,89,helio,octa,2.00,0,1,...,391.64,NaN,NaN,NaN,NaN,1,512.0,7,2,3
1,Realme Narzo 80x 5G (8GB RAM + 128GB),https://www.smartprix.com/mobiles/realme-narzo...,14499.0,4.05,80,dimensity,octa,2.50,0,0,...,391.64,2.0,1.0,50.0,8.0,1,1024.0,3,2,3
2,OnePlus 5,https://www.smartprix.com/mobiles/oneplus-5-p1...,22999.0,4.40,74,snapdragon,octa,2.45,0,1,...,400.53,2.0,1.0,20.0,16.0,0,0.0,6,0,3
3,Samsung Galaxy J2 Pro (2018),https://www.smartprix.com/mobiles/samsung-gala...,7999.0,4.10,49,snapdragon,quad,1.40,0,0,...,220.29,1.0,1.0,8.0,5.0,1,1024.0,1,0,0
4,Xiaomi Redmi Note 10 (6GB RAM + 128GB),https://www.smartprix.com/mobiles/xiaomi-redmi...,14999.0,4.50,78,snapdragon,octa,2.20,1,0,...,409.30,4.0,1.0,48.0,13.0,1,512.0,4,2,2


In [11]:
camera_specs = []
display_specs = []

In [12]:
driver = uc.Chrome()
driver.maximize_window()

i = 0

while i < df.shape[0]:
    if i % 1000 == 0:
        driver.quit()
        time.sleep(2)
        driver = uc.Chrome()
        driver.maximize_window()
        
    link = df.iloc[i]['link']
    driver.get(link)
    feature_box = driver.find_element(By.CSS_SELECTOR, ".sm-box.padded.pg-prd-h-feat")
    features = feature_box.find_elements(By.TAG_NAME, "li")
    camera_info = np.nan
    for feature in features:
        text = feature.text.lower()
        if 'camera' in text:
            camera_info = text
            break
    camera_specs.append(camera_info)
    
    try:
        dispay_section = driver.find_elements(By.CLASS_NAME, "group")[1]
        display_info = dispay_section.find_element(By.TAG_NAME, "li").text
    except:
        display_specs.append(np.nan)
    else:
        display_specs.append(display_info)
    
    
    i += 1
            
    time.sleep(1)
    

driver.quit()

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=61159): Read timed out. (read timeout=120)

In [9]:
print(df.iloc[2]['link'])

https://www.smartprix.com/mobiles/oneplus-5-p1101pdglcaa


In [19]:

# camera_specs = []
# display_specs = []

def start_browser():
    driver = uc.Chrome()
    driver.set_page_load_timeout(50)  # seconds
    driver.maximize_window()
    return driver

driver = start_browser()

for i in range(df.shape[0]):
    if i % 10 == 0 and i != 0:
        print(f"Restarting browser at iteration {i}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()

    link = df.iloc[i]['link']
    
    try:
        driver.get(link)
    except TimeoutException:
        print(f"[{i}] Timeout while loading: {link}")
        camera_specs.append(np.nan)
        display_specs.append(np.nan)
        continue
    except Exception as e:
        print(f"[{i}] Unexpected error loading page: {e}")
        camera_specs.append(np.nan)
        display_specs.append(np.nan)
        continue

    # Camera info
    camera_info = np.nan
    try:
        feature_box = driver.find_element(By.CSS_SELECTOR, ".sm-box.padded.pg-prd-h-feat")
        features = feature_box.find_elements(By.TAG_NAME, "li")
        for feature in features:
            text = feature.text.lower()
            if 'camera' in text:
                camera_info = text
                break
    except NoSuchElementException:
        print(f"[{i}] Camera info not found.")
    except Exception as e:
        print(f"[{i}] Error finding camera info: {e}")

    camera_specs.append(camera_info)

    # Display info
    display_info = np.nan
    try:
        groups = driver.find_elements(By.CLASS_NAME, "group")
        if len(groups) > 1:
            display_info = groups[1].find_element(By.TAG_NAME, "li").text
    except Exception as e:
        print(f"[{i}] Display info not found or error: {e}")
    
    display_specs.append(display_info)

    print(f"[{i}] Scraped successfully.")
    time.sleep(1)

driver.quit()


[0] Scraped successfully.
[1] Scraped successfully.
[2] Scraped successfully.
[3] Scraped successfully.
[4] Scraped successfully.
[5] Scraped successfully.
[6] Scraped successfully.
[7] Scraped successfully.
[8] Scraped successfully.
[9] Scraped successfully.
Restarting browser at iteration 10
[10] Scraped successfully.
[11] Scraped successfully.
[12] Scraped successfully.
[13] Scraped successfully.
[14] Scraped successfully.
[15] Scraped successfully.
[16] Scraped successfully.
[17] Scraped successfully.
[18] Scraped successfully.
[19] Scraped successfully.
Restarting browser at iteration 20
[20] Scraped successfully.
[21] Scraped successfully.
[22] Scraped successfully.
[23] Scraped successfully.
[24] Scraped successfully.
[25] Scraped successfully.
[26] Scraped successfully.
[27] Scraped successfully.
[28] Scraped successfully.
[29] Scraped successfully.
Restarting browser at iteration 30
[30] Scraped successfully.
[31] Scraped successfully.
[32] Scraped successfully.
[33] Scraped s

KeyboardInterrupt: 

In [ ]:
# Our HTML connection got aborted after scraping the details of first 553 phones.
# So, modifying our above code and restarting the scraping from 554th phone.

In [24]:
len(display_specs)

558

In [28]:
# display_specs = display_specs[:-5]
# camera_specs = camera_specs[:-5]

In [37]:
def start_browser():
    driver = uc.Chrome()
    driver.set_page_load_timeout(50)  # seconds
    driver.maximize_window()
    return driver

driver = start_browser()
i = 553
while i < df.shape[0]:
    if i % 10 == 0 and i != 0:
        print(f"Restarting browser at iteration {i}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()

    link = df.iloc[i]['link']
    
    try:
        driver.get(link)
    except TimeoutException:
        print(f"[{i}] Timeout while loading: {link}")
        camera_specs.append(np.nan)
        display_specs.append(np.nan)
        i += 1
        continue
    except Exception as e:
        print(f"[{i}] Unexpected error loading page: {e}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()
        continue

    # Camera info
    camera_info = np.nan
    try:
        feature_box = driver.find_element(By.CSS_SELECTOR, ".sm-box.padded.pg-prd-h-feat")
        features = feature_box.find_elements(By.TAG_NAME, "li")
        for feature in features:
            text = feature.text.lower()
            if 'camera' in text:
                camera_info = text
                break
    except NoSuchElementException:
        print(f"[{i}] Camera info not found.")
    except Exception as e:
        print(f"[{i}] Error finding camera info: {e}")

    camera_specs.append(camera_info)

    # Display info
    display_info = np.nan
    try:
        groups = driver.find_elements(By.CLASS_NAME, "group")
        if len(groups) > 1:
            display_info = groups[1].find_element(By.TAG_NAME, "li").text
    except Exception as e:
        print(f"[{i}] Display info not found or error: {e}")
    
    display_specs.append(display_info)

    print(f"[{i}] Scraped successfully.")
    time.sleep(1)
    i += 1

driver.quit()

[553] Scraped successfully.
[554] Scraped successfully.
[555] Scraped successfully.
[556] Scraped successfully.
[557] Scraped successfully.
[558] Scraped successfully.
[559] Scraped successfully.
Restarting browser at iteration 560
[560] Scraped successfully.
[561] Scraped successfully.
[562] Scraped successfully.
[563] Scraped successfully.
[564] Scraped successfully.
[565] Scraped successfully.
[566] Scraped successfully.
[567] Scraped successfully.
[568] Scraped successfully.
[569] Scraped successfully.
Restarting browser at iteration 570
[570] Scraped successfully.
[571] Scraped successfully.
[572] Scraped successfully.
[573] Scraped successfully.
[574] Scraped successfully.
[575] Scraped successfully.
[576] Scraped successfully.
[577] Scraped successfully.
[578] Scraped successfully.
[579] Scraped successfully.
Restarting browser at iteration 580
[580] Scraped successfully.
[581] Scraped successfully.
[582] Scraped successfully.
[583] Scraped successfully.
[584] Scraped successful

In [38]:
df.shape[0]

4245

In [39]:
len(camera_specs)

4245

In [40]:
len(display_specs)

4245

In [45]:
temp_df = pd.DataFrame({
    'camera_specs' : camera_specs,
    'display_specs' : display_specs
})

In [83]:
temp_df.to_csv("smartphone_extra_details.csv", index = False)